<a href="https://colab.research.google.com/github/venkatasl/AIML_TRAINING_VENKAT/blob/main/Gradient_descent_and_back_propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent Tutorial

Link to Presentation: https://docs.google.com/presentation/d/1basYOmB3uW8l-p-v1OChzzYI0bzm-r3dr8wFTRpluhk/edit?usp=sharing

## Import Libraries

In [1]:
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

## Calculating the gradients of functions

Let us start with a simple function: sum of squares. Its gradient is known as 2*x

In [2]:
def f1(X):
    return torch.sum(torch.square(X))


def grad_f1(X):
    grad = torch.zeros_like(X)
    for i in range(len(X)):
        grad[i] = 2*X[i]
    return grad


Let us see what our function does

In [3]:
x = torch.tensor([1,2,3]);
print('Original X: ', x)
print('f1(x): ',f1(x))
print('Calculated d(f1)/d(x): ',grad_f1(x))

Original X:  tensor([1, 2, 3])
f1(x):  tensor(14)
Calculated d(f1)/d(x):  tensor([2, 4, 6])


We don't need to calculate the gradient function by ourselves. Pytorch provides **autograd** functionality which calculates the gradients for us on the fly. To do that, we need to define a Pytorch tensor with the flag **requires_grad=True**. We also need to make sure all the functions we are applying on the tensor is done using pytorch functions.

For all tensors with requires_grad=True, Pytorch maintains a graph of the operations so that the gradient can be calculated when required, using **backward** function.

In [4]:
x = torch.tensor([4,5,6], dtype=float, requires_grad=True);
print('Original X: ', x)
y = f1(x)
print('f1(x): ', y)
dy = grad_f1(x)
print('Calculated gradient dy/dx: ', dy.data)
# calculate the gradients using autograd
y.backward()
# now dy/dx will be present in x.grad
print('Autograd dy/dx: ', x.grad)

Original X:  tensor([4., 5., 6.], dtype=torch.float64, requires_grad=True)
f1(x):  tensor(77., dtype=torch.float64, grad_fn=<SumBackward0>)
Calculated gradient dy/dx:  tensor([ 8., 10., 12.], dtype=torch.float64)
Autograd dy/dx:  tensor([ 8., 10., 12.], dtype=torch.float64)


Let us try a more complex function with multiple steps:
z = (x^2 + y)^3

Let us calculate by hand. Let us break it like this:
w = x^2 + y
z = w^3

dz/dw = 3w^2
dw/dx = 2x
dw/dy = 1
dz/dx = dz/dw*dw/dx  etc.

In [ ]:
# input
x = torch.tensor([2], dtype=float, requires_grad=True)
print('original x: ', x)
y = torch.tensor([5], dtype=float, requires_grad=True)
print('original y: ', y)

# calculate our function
def f2(x,y):
  w = x*x + y
  z = w**3
  return w, z

w,z = f2(x,y)
print('intermediate w: ', w.data)
print('z: ', z.data)

Let us check the gradients computed by Pytorch for the w function:

In [ ]:
w.backward()
print('dw/dx : ', x.grad)
print('dw/dy : ', y.grad)

Let us calculate the gradients for the z function:

In [ ]:
z.backward()

In Pytorch, we can't run **backward()** through the same graph twice to save memory. So we have to run the function again or put the flag **retain_graph=True**

In [ ]:
# we need to clear all previous gradients, otherwise, it will get added up
x = x.detach()
y = y.detach()
x.requires_grad=True
y.requires_grad=True


w,z = f2(x,y)

z.backward()

print('dz/dx = dz/dw * dw/dx: ', x.grad)
print('dz/dy = dz/dw * dw/dy: ', y.grad)

## Questions
* Try with your own functions of multiple steps and multiple variables. Does the calculated gradient match the actual gradient?
* How do we calculate the gradient of a vector?

# Gradient Descent

Let us try to find the minimum of a simple function x^2. First let us look at the graph of the function.

In [ ]:
def f3(x):
  return x**2

def plot_function(function, start, end):
  x = torch.linspace(start, end, 1000)
  y = f3(x)
  plt.plot(x,y)




In [ ]:
plot_function(f3, -1000,1000)
plt.show()

Here is a function for **Gradient Descent**, where we start with a random **x**, and move slowly in the negative gradient direction

**Note:** Talk about convergence_threshold
- Just a bargain between time and precision
- Near the minima, update rate becomes very slow
- Also, getting an exact match for complex functions is difficult (splly when going stochastic)

In [ ]:
def gradient_descent_visualize(function, lr=0.1, convergence_threshold = 0.01, max_iter = 10000):
  x = torch.rand(1, requires_grad=True)
  x_history = []
  for ii in range(max_iter):
    y = function(x)
    y.backward()
    x_new = x - x.grad*lr
    if torch.abs(x_new - x) < convergence_threshold:
      x = x_new
      break
    # store the history of steps so that we can see
    x_history = x_history + [x_new]
    # zero the gradients
    x = x_new.detach()
    x.requires_grad = True

  return x, torch.tensor(x_history)


In [ ]:
x, history = gradient_descent_visualize(f3 , lr=0.1)
print(x)

Let us write a function for visualizing the steps taken

In [ ]:
def plot_function_2(function, history):
  y = function(history)
  plt.plot(history, y,color='red', marker='o', fillstyle='full')

In [ ]:
plot_function(f3, -1, 1)
plot_function_2(f3, history)
plt.show()

## Questions
1. Try with different iterations, learning rates and functions. Try non-convex functions and see how it converges!
2. Does changing these values give you better minima estimation?

# Doing it for an MLP!

Let us try to apply the gradient descent algorithm to a multi layer perceptron. The steps are the same. The increased complexity of the function doesn't matter as Autograd calculates the gradient for us.


For a neural network, there are two inputs to the function: f(x, w) where x is the input (image, text etc. ) and w is the weight. The neural network will have a loss function: L(f(x, w)) or L(x, w). We will be doing gradient descent on w using dL/dw and not on x, as we can't control the input.

Instead of using a single learning rate and updating the weights ourselves, Pytorch provides 'optimizers' which implement many algorithmic tricks, like variable learning rates, momentum, weight decay etc. When running gradient descent for an MLP, it is better to use an optimizer instead of manually changing the weights.


The following code is taken from the lab on Day 1

### Defining a simple convolutional model

In [ ]:
# CNN with 2 CONV layers and 3 FC layers
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 128)
        # output layer 10 classes
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flatten all dimensions except batch
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Take care of the data

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Classes in CIFAR10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print('Training data shape : ', trainset.data.shape, len(trainset.targets))
print('Testing data shape : ', testset.data.shape, len(testset.targets))

# Find the unique numbers from the train labels
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Let us write a training function to implement gradient descent:

In [ ]:
def train(num_epochs, model, train_loader, loss_func, optimizer):

  # Training mode
  model.train()

  train_losses = []
  train_acc = []

  # Train the model
  for epoch in range(num_epochs):
    running_loss = 0
    running_acc = 0
    for i, (images, labels) in enumerate(train_loader):

      # clear gradients for this training step
      optimizer.zero_grad()

      # Forward pass
      output = model(images)

      # Calculate loss
      loss = loss_func(output, labels)

      # Backpropagation, compute gradients
      loss.backward()

      # Apply gradients
      optimizer.step()

      # Running loss
      running_loss += loss.item()

      # indices of max probabilities
      _, preds = torch.max(output, dim=1)

      # Calculate number of correct predictions
      correct = (preds.float() == labels).sum()
      running_acc += correct

      # Average loss and acc values
      epoch_loss = running_loss / len(train_loader.dataset)
      epoch_acc = running_acc / len(train_loader.dataset)

    train_losses.append(epoch_loss)
    train_acc.append(epoch_acc)
    print ('Epoch {}/{}, Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch + 1, num_epochs, epoch_loss, epoch_acc*100))

  return train_losses, train_acc

Test function

In [ ]:
def test_model(model, testloader):
  correct = 0
  total = 0
  # Deactivate autograd engine (don't compute grads since we're not training)
  with torch.no_grad():
    for data in testloader:
        images, labels = data
        # Calculate outputs by running images through the network
        outputs = model(images)
        # The class with the highest value is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

  print('Accuracy of the network: %d %%' % (
      100 * correct / total))

### Do the training!

In [ ]:
# start with a fresh model
model = Net()
print(model)

We need a single value to do backward() on. Thus, we need to use the appropriate loss. The common loss function for classification is cross entropy loss.

In [ ]:
# Cross Entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

Let us pick one of the many optimizers available in Pytorch. Here we are using SGD. You can substitute with any optimizer from this page: https://pytorch.org/docs/stable/optim.html#per-parameter-options

In [ ]:
# SGD optimizer with momentum
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

Here is the training code:

In [ ]:
num_epochs = 5  # iterations
train_losses, train_acc = train(num_epochs, model, trainloader, criterion, optimizer)

In [ ]:
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(1,2, 1)
ax.plot(np.arange(1,len(train_losses)+1),train_losses)
plt.xlabel('Training loss')
plt.ylabel('Epochs')
ax.set_title('Loss vs Epochs')
ax = fig.add_subplot(1,2, 2)
ax.plot(np.arange(1,len(train_acc)+1),train_acc)
plt.xlabel('Training accuracy')
plt.ylabel('Epochs')
ax.set_title('Accuracy vs Epochs')
plt.show()

In [ ]:
# Accuracy on test data after training
test_model(model, testloader)

### Questions
1. Try with different hyperparameters of learning rate and momentum
2. Try with different optimizer values.